# Batch segmentation with text prompts


## Enable Jupyter Lab Leaflet

Run the following cell and reload  the page to enable the Jupyter Lab Leaflet extension.

In [ ]:
!jupyter labextension enable jupyter-leaflet

In [25]:
import leafmap
from samgeo import tms_to_geotiff, split_raster, SamGeo
from samgeo.text_sam import LangSAM

## Create an interactive map

In [12]:
m = leafmap.Map(center=[60.89650, -162.4580], zoom=17, height="800px")
m.add_basemap("SATELLITE")
m

Map(center=[60.8965, -162.458], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

## Download a sample image

Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

You can also use your own image. Uncomment and run the following cell to use your own image.

In [4]:
image = '/corral-repl/tacc/aci/PT2050/projects/PTDATAX-209/Aerial/Nunap.tif'

Display the downloaded image on the map.

In [13]:
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m

Map(bottom=9575162.0, center=[60.894813, -162.458614], controls=(ZoomControl(options=['position', 'zoom_in_tex…

## Split the image into tiles

In [22]:
split_raster(image, out_dir="tiles", tile_size=(3000, 3000), overlap=10)

## Initialize LangSAM class

The initialization of the LangSAM class might take a few minutes. The initialization downloads the model weights and sets up the model for inference.

In [7]:
sam = LangSAM()

final text_encoder_type: bert-base-uncased


## Specify text prompts

In [8]:
text_prompt = "building structures"

## Segment images

Part of the model prediction includes setting appropriate thresholds for object detection and text association with the detected objects. These threshold values range from 0 to 1 and are set while calling the predict method of the LangSAM class.

`box_threshold`: This value is used for object detection in the image. A higher value makes the model more selective, identifying only the most confident object instances, leading to fewer overall detections. A lower value, conversely, makes the model more tolerant, leading to increased detections, including potentially less confident ones.

`text_threshold`: This value is used to associate the detected objects with the provided text prompt. A higher value requires a stronger association between the object and the text prompt, leading to more precise but potentially fewer associations. A lower value allows for looser associations, which could increase the number of associations but also introduce less precise matches.

Remember to test different threshold values on your specific data. The optimal threshold can vary depending on the quality and nature of your images, as well as the specificity of your text prompts. Make sure to choose a balance that suits your requirements, whether that's precision or recall.

In [23]:
sam.predict_batch(
    images="tiles",
    out_dir="masks",
    text_prompt=text_prompt,
    box_threshold=0.24,
    text_threshold=0.24,
    mask_multiplier=1,
    dtype="uint8",
    merge=True,
    verbose=True,
)

Processing image 01 of 56: tiles/tile_0_0.tif...
Processing image 02 of 56: tiles/tile_0_1.tif...
Processing image 03 of 56: tiles/tile_0_2.tif...
Processing image 04 of 56: tiles/tile_0_3.tif...
Processing image 05 of 56: tiles/tile_0_4.tif...
Processing image 06 of 56: tiles/tile_0_5.tif...
Processing image 07 of 56: tiles/tile_0_6.tif...
Processing image 08 of 56: tiles/tile_1_0.tif...
Processing image 09 of 56: tiles/tile_1_1.tif...
Processing image 10 of 56: tiles/tile_1_2.tif...
Processing image 11 of 56: tiles/tile_1_3.tif...
Processing image 12 of 56: tiles/tile_1_4.tif...
Processing image 13 of 56: tiles/tile_1_5.tif...
Processing image 14 of 56: tiles/tile_1_6.tif...
Processing image 15 of 56: tiles/tile_2_0.tif...
Processing image 16 of 56: tiles/tile_2_1.tif...
Processing image 17 of 56: tiles/tile_2_2.tif...
Processing image 18 of 56: tiles/tile_2_3.tif...
Processing image 19 of 56: tiles/tile_2_4.tif...
Processing image 20 of 56: tiles/tile_2_5.tif...
Processing image 21 

## Visualize the results

In [26]:
mask = "masks/merged.tif"
shapefile = "Nunap.shp"
geosam = SamGeo(
    model_type="vit_h",
    checkpoint="sam_vit_h_4b8939.pth",
    sam_kwargs=None,
)
geosam.tiff_to_vector(mask, shapefile)

In [29]:
vector = "segment.gpkg"
geosam.tiff_to_gpkg(mask, vector, simplify_tolerance=None)

In [30]:
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.5,
}
m.add_vector(vector, layer_name="Vector", style=style)
m

Map(bottom=4789702.0, center=[60.876444114650866, -162.4194180965424], controls=(ZoomControl(options=['positio…